<a href="https://colab.research.google.com/github/andartya-setyawan/TubesAnalisaKeputusanTI/blob/main/Analisis_Pasar_Mobil_Australia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tugas Besar: Analisis Pasar Mobil Australia

Nama: Andartya Setyawan Darna<br>
NIM: 202022420016
---
Proyek ini berfokus pada analisis data pasar mobil di Australia (Australia Car Market Data). Dataset ini berisi ribuan entri mengenai berbagai jenis kendaraan yang dijual, mencakup berbagai merek, model, dan spesifikasi teknis lainnya. Tujuan utama dari analisis ini adalah memanfaatkan teknik Machine Learning untuk memprediksi kategori harga mobil (apakah termasuk kategori High-End/Mahal atau Budget/Terjangkau) berdasarkan karakteristik kendaraan tersebut.

Dengan memanfaatkan dataset yang mencakup berbagai informasi penting seperti merek, tahun pembuatan, tipe transmisi, tipe bodi, jarak tempuh, dan spesifikasi mesin, kami bertujuan untuk mengidentifikasi pola dan wawasan yang dapat memberikan pemahaman mendalam mengenai valuasi kendaraan. Melalui data preprocessing yang komprehensif, analisis data eksploratif (Exploratory Data Analysis/EDA), dan penerapan model klasifikasi Random Forest, proyek ini berupaya tidak hanya untuk memprediksi kategori harga dengan akurasi tinggi, tetapi juga mengungkap faktor-faktor mendasar yang paling berpengaruh terhadap penentuan harga mobil di pasaran.

Dengan memanfaatkan kemampuan bahasa pemrograman Python, library pandas untuk manipulasi data, matplotlib dan seaborn untuk visualisasi, serta Scikit-Learn sebagai backend untuk pemodelan, kita akan menavigasi proses analisis data secara menyeluruh. Perjalanan ini akan membawa kita dari pemuatan data awal (data loading) dan pembersihan data (data cleaning) menuju pembangunan dan evaluasi model, dengan tujuan akhir memberikan wawasan prediktif yang berharga dalam domain pasar otomotif.